# Dude where's my car

![alt Dude where's my car](https://64.media.tumblr.com/tumblr_lq1wrnrALQ1r1uduuo1_500.gif))

So I had to return my nice company car, a Volkswagen ID.4



Now I have switched to a new position, and therefore I have to depart with this really nice car.

I know, that all (or most) of the used company cares are sold through [Semler Mobility](https://www.semlermobility.dk/), so I was wondering, when my car is up for sale. To find out, let's dig down in some of the API's behind, and use Python to keep a track on, when my particular car, might turn up on the sales listing, so let's get started

## Import the needed libraries
In this scenario I have choosen to use `httpx` library as well as `Pandas Dataframe`


In [ ]:
import httpx
import json
import pandas as pd

# Next make a definition to loop and fecth the data
def fetch_data(skip, page_size):
    url = "https://www.semlermobility.dk/umbraco/api/ProductSearchApi/Search"
    params = {
        "nodeId": 1248,
        "sortBy": "popularity",
        "skip": skip,
        "pageSize": page_size,
        "culture": "da-DK",
        "brands": "volkswagen",
        "models": "ID.4",
        "searchLocation": "Vehicles Page"
    }
    #print(f"Fetching data from: {url} with params: {params}")
    response = httpx.get(url, params=params)
    data = response.json()
    #print(f"Data fetched for skip={skip}")
    return data

# Initialize variables
all_results = []
page_size = 100
skip = 0

# Fetch the first page to get the total number of results
data = fetch_data(skip, page_size)
total_results = data["totalResults"]
print(f"Total results: {total_results}")

# Loop to fetch all pages
while skip < total_results:
    data = fetch_data(skip, page_size)
    all_results.extend(data["results"])
    skip += page_size
    #print(f"Processed {skip} results")

# Load all results into a pandas DataFrame
df = pd.DataFrame(all_results)

# Columns to remove
columns_to_remove = ['imageUrl', 'hasImageUrl', 'isSlide', 'brandAlias', 'brandLogoUrl', 'isCommercial', 'registrationDate', 'secondaryDealerKey', 'hasBrandLogoUrl', 'hasClockedKilometers', 'hasModelYear', 'hasSegment', 'hasRegistration']

# Remove the specified columns
df = df.drop(columns=columns_to_remove)

# Display the DataFrame
#print("Final DataFrame:")
#print(df)

# Define the specific VIN number you are looking for
specific_vin = "WVGZZZE26RE026755"

# Search for the specific VIN in the DataFrame
result = df.loc[df['vin'] == specific_vin]

# Display the result
if not result.empty:
    print(f"Dude, I found your car: {result['url'].iloc[0]}")
else:
    print(f"Did not find your car among the {total_results} VW ID.4 cars currently for sale...")      

